In [28]:
#載入LineBot所需要的模組
from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import *
#訊息比對
import re 
#use imgur
from imgurpython import ImgurClient

In [3]:
app = Flask(__name__)

# 必須放上自己的Channel Access Token
line_bot_api = LineBotApi('plMcM04uPLPHEsBYPqZZtFHj0pM9I19dOVByC5fp7FZf4gFTW83BZVWYBcTW/g/ENqoWfTrIUx6tLCyAVLRuGh4At1PXF2GZPqhTvswClAZCphOFNRgDyAvobPNymbm4D04ZsOAxaq8d9wNT6ojOxwdB04t89/1O/w1cDnyilFU=')

# 必須放上自己的Channel Secret
handler = WebhookHandler('fa28c66980cf43dd8b1e41630b2f8fd1')

#伺服器檢測
line_bot_api.push_message('U82e65d3ed7880828c259a6af649abc96', TextSendMessage(text='你可以開始了'))

In [4]:
# 監聽所有來自 /callback 的 Post Request
@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']
    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [31]:
#訊息傳遞區塊
##### 基本上程式編輯都在這個function #####
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    #活動顯示
    print(event)
    
    message = event.message.text #原TextSendMessage(text=event.message.text)
    if re.match("使用說明",message):
        line_bot_api.reply_message(event.reply_token,TextSendMessage("才不告訴你勒~~"))
    if re.match('1',message):
        image_message = ImageSendMessage(
            original_content_url='https://media.nownews.com/nn_media/thumbnail/2019/10/1570089924-27a9b9c9d7facd3422fe4610dd8ebe42-696x386.png',
            preview_image_url='https://media.nownews.com/nn_media/thumbnail/2019/10/1570089924-27a9b9c9d7facd3422fe4610dd8ebe42-696x386.png'
            )
        line_bot_api.reply_message(event.reply_token, image_message)
    if re.match('2',message):
        video_message = VideoSendMessage(
            original_content_url='https://i.imgur.com/XVmZmIE.mp4',
            preview_image_url='https://img.ttshow.tw/images/media/frontcover/2020/08/06/6.jpg'
        )
        line_bot_api.reply_message(event.reply_token, video_message)
    if re.match('3',message):
        audio_message = AudioSendMessage(
            original_content_url='https://cdn.voicetube.com/everyday_records/5664/1626443219.mp3',duration=30000
        )
        line_bot_api.reply_message(event.reply_token, audio_message)
    if "股票 " in message:
        buttons_template_message = TemplateSendMessage(
            alt_text = "股票資訊",template=CarouselTemplate(
                columns=[
                    CarouselColumn(
                        thumbnail_image_url ="https://www.cwb.gov.tw/Data/radar/CV1_3600.png",
                        title = message + " 股票資訊",
                        text ="請點選想查詢的股票資訊",
                        actions =[
                            MessageAction(
                                label= message[3:] + " 個股資訊",
                                text= "個股資訊 " + message[3:]
                            ),
                            MessageAction(
                                    label= message[3:] + " 個股新聞",
                                    text= "個股新聞 " + message[3:]
                            )
                        ]
                    )
                ]
            )
        )
        line_bot_api.reply_message(event.reply_token, buttons_template_message)
    else:
        line_bot_api.reply_message(event.reply_token,message)
    
    

In [32]:
@handler.add(MessageEvent, message=ImageMessage)
def handle_message(event):
    message = event.message.image
    if (event.message.type == "image"):
        SendImage = line_bot_api.get_message_content(event.message.id)
        print(SendImage)

In [33]:
#主程式
import os 
if (__name__ == "__main__"):
    port = int(os.environ.get('PORT', 400))
    app.run(host='0.0.0.0', port=port)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.0.103:400/ (Press CTRL+C to quit)
192.168.0.103 - - [25/Oct/2022 00:03:42] "GET / HTTP/1.1" 404 -
192.168.0.103 - - [25/Oct/2022 00:03:42] "GET /favicon.ico HTTP/1.1" 404 -
192.168.0.103 - - [25/Oct/2022 00:03:46] "GET / HTTP/1.1" 404 -


{"deliveryContext": {"isRedelivery": false}, "message": {"id": "17007226805030", "text": "2", "type": "text"}, "mode": "active", "replyToken": "85c99b05103c46698f711ba4151806b3", "source": {"type": "user", "userId": "U82e65d3ed7880828c259a6af649abc96"}, "timestamp": 1666627517201, "type": "message", "webhookEventId": "01GG5BVF8AA5C62KY6VYHMW4E1"}


[2022-10-25 00:05:17,834] ERROR in app: Exception on /callback [POST]
Traceback (most recent call last):
  File "c:\Users\Z4862\anaconda3\envs\line_bot\lib\site-packages\flask\app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\Z4862\anaconda3\envs\line_bot\lib\site-packages\flask\app.py", line 1518, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\Users\Z4862\anaconda3\envs\line_bot\lib\site-packages\flask\app.py", line 1516, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\Z4862\anaconda3\envs\line_bot\lib\site-packages\flask\app.py", line 1502, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "C:\Users\Z4862\AppData\Local\Temp\ipykernel_5536\805643118.py", line 12, in callback
    handler.handle(body, signature)
  File "c:\Users\Z4862\anaconda3\envs\line_bot\lib\site-packages\linebot\webhook.py", line 259, in handle
    self.__invoke

{"deliveryContext": {"isRedelivery": false}, "message": {"id": "17007227402517", "text": "1", "type": "text"}, "mode": "active", "replyToken": "cd21fef4f60e4aecb7dd832cc4b01a1b", "source": {"type": "user", "userId": "U82e65d3ed7880828c259a6af649abc96"}, "timestamp": 1666627528753, "type": "message", "webhookEventId": "01GG5BVT8AXDJXZ34T30A3DB0J"}
